In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("hive-queries") \
    .master("spark://spark-master:7077")\
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# spark.stop()

In [12]:
#Use WES Database
spark.sql("USE wes")
current_db = spark.sql("SELECT current_database()").collect()[0][0]
print(current_db)

wes


In [3]:
from utils import run_spark_sql

In [11]:
run_spark_sql("1_energy_overview.sql","1_energy_overview.csv")

+----+----------------+
|year|PRIM_ENERGY_CONS|
+----+----------------+
|1990|        94762.63|
|1991|        95531.11|
|1992|        96604.67|
|1993|        97426.49|
|1994|        98706.89|
|1995|       101034.98|
|1996|       103924.71|
|1997|       104994.93|
|1998|       105593.14|
|1999|       107513.06|
|2000|       110397.99|
|2001|       111476.55|
|2002|       113880.28|
|2003|        117901.2|
|2004|       123710.62|
|2005|       127836.38|
|2006|       131628.22|
|2007|       135648.38|
|2008|       137263.93|
|2009|       134974.75|
+----+----------------+
only showing top 20 rows



In [20]:
run_spark_sql("2_energy_consumption_top15.sql","2_energy_consumption_top15.csv")

+--------------+--------+--------------------------+----+
|       country|iso_code|primary_energy_consumption|RANK|
+--------------+--------+--------------------------+----+
|         China|     CHN|                  43873.07|   1|
| United States|     USA|                 25945.025|   2|
|         India|     IND|                  9584.976|   3|
|        Russia|     RUS|                   8745.49|   4|
|         Japan|     JPN|                  4983.889|   5|
|        Canada|     CAN|                  3843.694|   6|
|        Brazil|     BRA|                   3570.67|   7|
|       Germany|     DEU|                  3549.456|   8|
|   South Korea|     KOR|                  3489.739|   9|
|          Iran|     IRN|                  3360.935|  10|
|  Saudi Arabia|     SAU|                  2987.546|  11|
|        France|     FRA|                   2611.23|  12|
|        Mexico|     MEX|                  2219.266|  13|
|     Indonesia|     IDN|                  2154.942|  14|
|United Kingdo

In [18]:
run_spark_sql("2_energy_consumption_pct_top15.sql", "2_energy_consumption_pct_top15.csv") 

+------------------------------------------------------------------+
|round(((sum(primary_energy_consumption) / total_energy) * 100), 2)|
+------------------------------------------------------------------+
|                                                             74.17|
+------------------------------------------------------------------+



In [17]:
run_spark_sql("2_energy_consumption_pct_rem.sql", "2_energy_consumption_pct_rem.csv") 

+--------+
|count(1)|
+--------+
|     129|
+--------+



In [31]:
run_spark_sql("3_energy_breakdown_top15.sql", "3_energy_breakdown_top15.csv") 

+----+--------------------+---------------------+--------------------+------------------------+----------------------+------------------------+--------------------+----------------------+---------------------+
|year|perc_ren_consumption|perc_coal_consumption|perc_gas_consumption|perc_biofuel_consumption|perc_hydro_consumption|perc_nuclear_consumption|perc_oil_consumption|perc_solar_consumption|perc_wind_consumption|
+----+--------------------+---------------------+--------------------+------------------------+----------------------+------------------------+--------------------+----------------------+---------------------+
|1990|   6.603875227748031|   28.674992256085947|   19.93281372952673|     0.13608215427031167|     6.048725873619321|       6.778699498617638|   38.41397725982804|  0.001674441043321...| 0.013034787008694346|
|1991|   6.666393152711433|   28.233939151932958|  20.343556732027615|     0.13990238810267616|     6.090433489009183|      7.0374913576502065|   38.13846895774

In [7]:
run_spark_sql("4_electricity_gen_top15.sql", "4_electricity_gen_top15.csv") 


+--------------+----+------------------------+------------------+------------------------+--------------------+
|       country|year|total_fossil_electricity| total_consumption|non_renewable_percentage|renewable_percentage|
+--------------+----+------------------------+------------------+------------------------+--------------------+
|         China|2021|                 5623.99|          8242.446|                   68.23|               31.77|
| United States|2021|                 2512.39|           3446.46|                    72.9|                27.1|
|         India|2021|                 1337.63|1706.5900000000001|                   78.38|               21.62|
|         Japan|2021|                  680.58|            937.24|                   72.62|               27.38|
|        Russia|2021|                   666.3| 888.8499999999999|                   74.96|               25.04|
|        Brazil|2021|                  139.21|            705.48|                   19.73|              

In [18]:
run_spark_sql("4_electricity_share_top15.sql", "4_electricity_share_top15.csv") 

+--------------+----+-----------------+---------------------+------------------+----------------------+---------+--------------------+
|       country|year|fossil_share_elec|renewables_share_elec|electricity_demand|electricity_generation|max_share|      max_share_name|
+--------------+----+-----------------+---------------------+------------------+----------------------+---------+--------------------+
|         China|2021|           66.289|               28.908|           8466.32|               8484.02|   62.932|     coal_share_elec|
| United States|2021|           60.487|               20.743|           4192.93|               4153.62|   39.513|low_carbon_share_...|
|         India|2021|           78.053|               19.384|           1711.87|               1713.75|   74.173|     coal_share_elec|
|        Russia|2021|           60.008|               19.959|           1092.69|               1110.36|    41.99|      gas_share_elec|
|         Japan|2021|           71.002|               2

In [16]:
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
share_elec_columns_query = f"""
    SELECT
      country,
      electricity_generation,
      year,
      fossil_share_elec,
      renewables_share_elec
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    ORDER BY
      renewables_share_elec DESC,electricity_generation DESC ,renewables_share_elec  DESC
"""

# Running the query and displaying the results
share_elec_columns_data = spark.sql(share_elec_columns_query)
share_elec_columns_data.show()


+--------------------+----------------------+----+-----------------+---------------------+
|             country|electricity_generation|year|fossil_share_elec|renewables_share_elec|
+--------------------+----------------------+----+-----------------+---------------------+
|             Iceland|                 19.26|2021|              0.0|                100.0|
|              Bhutan|                   9.0|2021|              0.0|                100.0|
|             Albania|                  8.96|2021|              0.0|                100.0|
|               Nepal|                  6.12|2021|              0.0|                100.0|
|             Lesotho|                   0.5|2021|              0.0|                100.0|
|Central African R...|                  0.15|2021|              0.0|                100.0|
|            Ethiopia|                 14.69|2021|            0.068|               99.932|
|            Paraguay|                 40.02|2021|            0.175|               99.825|